**Хемоинформатика**

Предсказание противовирусной активности соединений - очевидная актуальная задача, которая позволит ускорить создание лекарств, используя современные цифровые инструменты. В рамках задачи необходимо собрать информацию о различных химических соединениях, для которых активность простив одного из вирусов (SARS-CoV-2) известна, а затем обучить модель для предсказания противовирусной активности. Для сбора подходит, например, база.


Dataset: CHEMBL391




# Библиотеки и файлы

In [7]:
!pip install rdkit
!pip install grakel
!pip install tensorflow scikit-learn xgboost dgl torch torchvision
!pip install dgl
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [8]:
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
import tensorflow as tf
from tensorflow.keras import layers, models
import torch
import torch.nn as nn
import torch.optim as optim
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors, Draw
from rdkit.Chem.Draw import IPythonConsole
from sklearn.utils import shuffle


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
data_monkey_act =  pd.read_csv('/content/drive/MyDrive/hac/data_monkey_act.csv',sep=';')


# Обработка данных

In [11]:
def train_procesing_for_cells(data_act):
  data_act.rename(columns={'Molecule ChEMBL ID':'ChEMBL ID'},inplace=True)
  data_act= data_act[data_act["Standard Type"] == "CC50"]
  data_act = data_act.dropna(subset=['Standard Value'])

  result = data_act.loc[data_act.groupby('ChEMBL ID')['Standard Value'].idxmax()]
  result = result[["Standard Value", "ChEMBL ID", "Smiles"]]
  return result

In [12]:
result_for_target_monkey = train_procesing_for_cells(data_monkey_act)
result_for_target_monkey.reset_index(inplace=True, drop=True)
result_for_target_monkey

,Standard Value,ChEMBL ID,Smiles
0,40.0,CHEMBL100319,Cc1cccc2ncc(CSCc3ccccc3)n12
1,13020.0,CHEMBL106562,COC1=CC(=O)c2ccccc2C1=O
2,50.0,CHEMBL1075790,CO[C@H]1[C@H](O)[C@H](O[C@H]2CC[C@@]3(C=O)[C@H...
3,1000000.0,CHEMBL1076929,CO[C@@H]1[C@H](OP(=O)(OC[C@H]2O[C@@H](n3cnc4c(...
4,1000000.0,CHEMBL1076930,CO[C@@H]1[C@H](OP(=O)(OC[C@H]2O[C@@H](n3cnc4c(...
...,...,...,...
6815,100000.0,CHEMBL98778,C#C[C@@]1(O)[C@H](O)[C@@H](CO)O[C@H]1n1ccc(N)n...
6816,200.0,CHEMBL99416,CCOC(=O)CSCc1cnc2cc(C)ccn12
6817,100.0,CHEMBL99533,C=Cc1cn(COC(CO)CO)c(=O)[nH]c1=O
6818,223100.0,CHEMBL999,C/C(O)=C(/C#N)C(=O)Nc1ccc(C(F)(F)F)cc1


In [60]:
data = result_for_target_monkey

In [14]:
#получение признаков из SMILES
def rdkit_fp(smiles_column: pd.Series, radius=3, nBits=2048, useChirality=False):
    # morganFP_rdkit
    def desc_gen(mol):
        mol = Chem.MolFromSmiles(mol)
        bit_vec = np.zeros((1,), np.int16)
        DataStructs.ConvertToNumpyArray(
            AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits, useChirality=useChirality), bit_vec)
        return bit_vec

    return pd.DataFrame.from_records(smiles_column.apply(func=desc_gen), columns=[f'bit_id_{i}' for i in range(nBits)])

def rdkit_2d(smiles_column: pd.Series):
    # 2d_rdkit
    descriptors = {i[0]: i[1] for i in Descriptors._descList}
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in smiles_column)

In [61]:
data = data.dropna()

In [16]:
Y = rdkit_fp(data['Smiles'])
Z = rdkit_2d(data['Smiles'])

Выходные данные были обрезаны до нескольких последних строк (5000).
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use MorganGenerator
[18:35:37] DEPRECATION WARNING: please use Morg

In [62]:
data = data.join(Y)
data = data.join(Z)

In [63]:
data = shuffle(data)
data = data[(data['Standard Value'] >= 0)]
data.reset_index(inplace=True, drop= True)
data=data.dropna()
y= data['Standard Value']
X = data.drop(['Smiles', 'Standard Value', "ChEMBL ID"], axis=1)


In [64]:
#стандартизация признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
joblib.dump(scaler, 'scaler_vero.pkl')


['scaler_vero.pkl']

In [65]:
# Удаление признаков с низкой дисперсией менее 1%
selector = VarianceThreshold(threshold=0.01)
X_var_thresh = selector.fit_transform(X_scaled)
joblib.dump(selector, 'selector_vero.pkl')


['selector_vero.pkl']

In [66]:
# Вычислим корреляционную матрицу
column_names = [f'feature_{i}' for i in range(X_var_thresh.shape[1])]
corr_matrix = np.corrcoef(X_var_thresh, rowvar=False)
upper_triangle = np.triu(np.abs(corr_matrix), k=1)

to_drop = [i for i in range(upper_triangle.shape[1]) if any(upper_triangle[:, i] > 0.95)]
removed_columns = [column_names[i] for i in to_drop]
X_uncorr = np.delete(X_var_thresh, to_drop, axis=1)

In [67]:
removed_columns

['feature_2049',
 'feature_2055',
 'feature_2056',
 'feature_2057',
 'feature_2060',
 'feature_2061',
 'feature_2064',
 'feature_2076',
 'feature_2077',
 'feature_2078',
 'feature_2079',
 'feature_2080',
 'feature_2081',
 'feature_2082',
 'feature_2083',
 'feature_2084',
 'feature_2085',
 'feature_2086',
 'feature_2087',
 'feature_2090',
 'feature_2091',
 'feature_2092',
 'feature_2093',
 'feature_2151',
 'feature_2153',
 'feature_2161',
 'feature_2162',
 'feature_2169',
 'feature_2172',
 'feature_2179',
 'feature_2181',
 'feature_2191',
 'feature_2204',
 'feature_2232']

In [68]:
# Сохраним 95% дисперсии данных
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_uncorr)
joblib.dump(pca, 'pca_model_vero.pkl')

['pca_model_vero.pkl']

In [69]:
y_log = np.log(y + 1e-6)
y_log

,Standard Value
0,11.430413
1,13.724491
2,12.658930
3,13.583579
4,12.899220
...,...
6809,10.126631
6810,12.206073
6811,10.126631
6812,11.512925


In [71]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_log, test_size=0.2, random_state=42)

# Обучение

In [1]:
from xgboost import XGBRegressor
xgb = XGBRegressor(
    learning_rate=0.05,
    max_depth=8,
    n_estimators=850,
    random_state=42
)

# 4. Обучение модели
xgb.fit(X_train, y_train)

# 5. Прогнозирование на тестовых данных
y_pred_xgb = xgb.predict(X_test)

# 6. Оценка модели
print("XGBoost Metrics:")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_xgb):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_xgb):.4f}")

XGBoost Metrics:
Mean Squared Error: 2.2350
Mean Absolute Error: 0.7603


Пробуемые модели

In [2]:
param_grid = {
    'iterations': [1000],  # Два значения для количества итераций
    'depth': [8],  # Два значения для глубины деревьев
    'learning_rate': [0.05],  # Два значения для скорости обучения
}

grid_search = GridSearchCV(estimator=CatBoostRegressor(verbose=0), param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшая модель
best_model = grid_search.best_estimator_
print("Лучшие параметры модели CatBoost:", grid_search.best_params_)

# 8. Прогнозирование с лучшей моделью
y_pred_best = best_model.predict(X_test)

# Оценка лучшей модели
mse_best = mean_squared_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f'\nЛучшие метрики модели:')
print(f'MSE: {mse_best:.4f}')
print(f'MAE: {mae_best:.4f}')



Лучшие метрики модели:
MSE: 2.6836
MAE: 0.8418
